In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# some imports from my project library
from turboflow.dataloaders import load_turbo2D_simple_numpy
from turboflow.utils import phy_utils as phy
from turboflow.utils import dsp_utils as dsp
from turboflow.utils import torch_utils as tch
from turboflow.utils import viz_utils as viz
from turboflow import evaluation as evl


# import torch lib
import torch
# import pl torch libs
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from turboflow.dataloaders import Turbo2DDataModule

In [3]:
dm = Turbo2DDataModule(train_ds=4, val_ds=4,test_ds=1,img_idx=42)
dm.prepare_data()

Y shape (64, 64, 2)
Y min, max: -2.365612 2.941536
after normalization, Y min, max: -0.8042097733972999 1.0
Y shape (64, 64, 2)
Y min, max: -2.365612 2.941536
after normalization, Y min, max: -0.8042097733972999 1.0
Y shape (256, 256, 2)
Y min, max: -2.382122 2.999472
after normalization, Y min, max: -0.7941804424245333 1.0


In [4]:
for batch in dm.train_dataloader():
    X, y = batch
    print(X.shape)
    print(y.shape)

for batch in dm.val_dataloader():
    X, y = batch
    print(X.shape)
    print(y.shape)
    print(X[:2,:])
    print(X[2,1]-X[2,0])
    
for batch in dm.test_dataloader():
    X, y = batch
    print(X.shape)
    print(y.shape)
    print(X[:2,:])
print('%1.8f' % (X[2,1]-X[2,0]))


torch.Size([4096, 2])
torch.Size([4096, 2])
torch.Size([4096, 2])
torch.Size([4096, 2])
tensor([[0.0000, 0.0000],
        [0.0000, 0.0157]])
tensor(0.0314)
torch.Size([65536, 2])
torch.Size([65536, 2])
tensor([[0.0000, 0.0000],
        [0.0000, 0.0039]])
0.00784314


In [12]:
from turboflow.models.phyrff_hard import plDivFreeRFFNet

args = {
    'name':'DivFreeNetRFF',
    'mlp_layers':[2] + [256]*3 + [1],
    'last_activation_fun':torch.nn.Tanh(),
    'do_rff' :   True, 
    'fft_nfeat' : 256,
    'fft_scale' :  10,
    'lam_pde': 1e-4,
    'lam_reg': 0,
    'lam_sfn': 0,
    'smallest_increment':0.00784314,
    'n_centers'   :30,
    'n_increments':4,
}

model = plDivFreeRFFNet(**args)

"do_rff":              True
"fft_nfeat":           256
"fft_scale":           10
"lam_pde":             0.0001
"lam_reg":             0
"lam_sfn":             0
"last_activation_fun": Tanh()
"mlp_layers":          [2, 256, 256, 256, 1]
"n_centers":           30
"n_increments":        4
"name":                DivFreeNet_RFF
"patch_dim":           32
"smallest_increment":  0.00784314


In [14]:
early_stop_callback = EarlyStopping(monitor='val_loss')

trainer = pl.Trainer(gpus=1, 
                     check_val_every_n_epoch=200, 
                     max_epochs=5000, 
                     callbacks=[early_stop_callback])

trainer.fit(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rff  | Fourier | 512   
1 | mlp  | MLP     | 263 K 
2 | div  | DivFree | 0     
---------------------------------
263 K     Trainable params
512       Non-trainable params
263 K     Total params
1.055     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/dicarlo_d/Documents/Code/TurboSuperResultion/venv/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
